# S&P500 Example: Hedging of an Asian Put Option with a Black Scholes-Hedge

First import the necessary packages

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from scipy.optimize import minimize
from Functions import*
from Functions_NonRobust import*

Only execute next box if program should run on a GPU

In [2]:
physical_devices = tf.config.list_physical_devices('GPU') 
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

Load the Data of 20 of the largest constituents

In [3]:
stocks =[]
stocks += [pd.read_csv("time_series/AAPL.OQ.csv")] #Apple Inc.
stocks += [pd.read_csv("time_series/MSFT.OQ.csv")] # Microsoft Corporation 
stocks += [pd.read_csv("time_series/AMZN.OQ.csv")] # Amazon.com Inc.
stocks += [pd.read_csv("time_series/GOOGL.OQ.csv")] #Alphabet Inc. Class C
stocks += [pd.read_csv("time_series/BRKb.N.csv")] # Berkshire Hathaway Inc. Class B
stocks += [pd.read_csv("time_series/JPM.N.csv")] #JPMorgan Chase & Co.
stocks += [pd.read_csv("time_series/JNJ.N.csv")] #Johnson & Johnsonjohn
stocks += [pd.read_csv("time_series/V.N.csv")] # Visa Inc. Class A
stocks += [pd.read_csv("time_series/UNH.N.csv")] #UnitedHealth Group Incorporated
stocks += [pd.read_csv("time_series/NVDA.OQ.csv")] #NVIDIA Corporation
stocks += [pd.read_csv("time_series/PG.N.csv")] #Procter & Gamble Company
stocks += [pd.read_csv("time_series/HD.N.csv")] #Home Depot Inc.
stocks += [pd.read_csv("time_series/MA.N.csv")] # Mastercard Incorporated Class A
stocks += [pd.read_csv("time_series/BAC.N.csv")] # Bank of America Corp
stocks += [pd.read_csv("time_series/DIS.N.csv")] # Walt Disney Company
stocks += [pd.read_csv("time_series/CMCSA.OQ.csv")] #Comcast Corporation Class A
stocks += [pd.read_csv("time_series/XOM.N.csv")] #Exxon Mobil Corporation
stocks += [pd.read_csv("time_series/ADBE.OQ.csv")] # Adobe Inc.
stocks += [pd.read_csv("time_series/VZ.N.csv")] #Verizon Communications Inc.
stocks += [pd.read_csv("time_series/INTC.OQ.csv")] # Intel Corporation


for i in range(len(stocks)):
    # Clean the NaNs
    stocks[i] = stocks[i].dropna()
    # Reset Indices
    stocks[i] = stocks[i].reset_index()
    # Only Dates and Close Values
    stocks[i]  = stocks[i] [["Date","CLOSE"]]

# Maxmimum Likelihood Estimation for all constituents

Define the function for the estimation.

In the Black Scholes - Case

$$
d X_t = (b_1X_t) dt + (a_1 X_t) dW_t,~t\geq 0,\\
X_0 = x_0
$$

we have a closed form solution:

$\hat{b_1} = \frac{1}{\Delta t_i(N-1)}\sum_{i=1}^{N-1} \log(x_{i+1}/x_i)$

$\hat{a_1} =  \frac{1}{\sqrt{\Delta t_i}} \cdot\sum_{i=1}^{N-1} \frac{(\log(x_{i+1}/x_i)-\hat{b_1})^2}{n}$

In [4]:
def compute_max_parameters(x, 
                           iterations = 1000,
                           initial_guess= [0.1,0.1],
                           tolerance = 1e-15,
                           Delta = 1/250,
                           method = 'COBYLA'):
    x_0 = np.array(initial_guess) 
    eps = tolerance # Tolerance to avoid that fractions and log-expressions become inf or -inf
    #Definte the Log-Likelihood Function
    def log_likelihood(param):
        a_1 = param[0]
        b_1 = param[1]
        constant = np.sqrt(2*np.pi*Delta)
        l= [-np.log((a_1*np.maximum(x[i],0))*constant+eps)-(1/(2*Delta))*((x[i+1]-x[i]-(b_1*x[i])*Delta)/(a_1*np.maximum(x[i],0)+eps))**2 for i in range(len(x)-1)]
        return -np.sum(l) # Mean instead of sum to have smaller values
    
    a1,b1 = minimize(log_likelihood,x_0,method=method,options={'maxiter': iterations,
                                                               'rhobeg':0.01},
                bounds = [(eps,None),(None,None)]).x
    a1 = np.max([a1,eps])
    return a1,b1

In [5]:
def compute_max_parameters_BS(x,Delta = 1/250):
    y = np.diff(np.log(x))
    b_1= np.mean(y)/Delta
    a_1 = (1/np.sqrt(Delta))*np.sum((y-b_1)**2)/len(y)
    return a_1, b_1

## Determination of MLE-Parameters

First in the crisis period

In [6]:
start_index = 2880
end_index = 2910
list_parameters_crisis=[]

for j in range(len(stocks)):
    #Create lists for the parameters
    list_a1 = []
    list_b1 = []
    list_a1_rescaled = []
    list_b1_rescaled = []
    
    #Compute optimal parameters
    i= 2879 # Begin of Crisis
    x = np.array(stocks[j].iloc[(i-250):i]["CLOSE"])
    a1,b1 = compute_max_parameters(x)        
    list_a1 += [a1]
    list_b1 += [b1]
        # Rescale the parameters
    list_parameters_crisis+= [pd.DataFrame({'a1': list_a1,
                   'b1': list_b1})]

C:\Users\julia\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\scipy\optimize\_minimize.py:540: RuntimeWarning: Method COBYLA cannot handle bounds.
  RuntimeWarning)


Then in the three periods thereafter

In [8]:
start_index = 2880
end_index = 2910
list_parameters=[]

for j in range(len(stocks)):
    #Create lists for the parameters
    list_a1 = []
    list_b1 = []
    list_a1_rescaled = []
    list_b1_rescaled = []
    
    #Compute optimal parameters
    for i in np.arange(2979,3275,100): # the additional periods
        x = np.array(stocks[j].iloc[(i-250):i]["CLOSE"])
        a1,b1 = compute_max_parameters(x)        
        list_a1 += [a1]
        list_b1 += [b1]
        # Rescale the parameters
    list_parameters+= [pd.DataFrame({'a1': list_a1,
                   'b1': list_b1})]

C:\Users\julia\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\scipy\optimize\_minimize.py:540: RuntimeWarning: Method COBYLA cannot handle bounds.
  RuntimeWarning)


# Testing

## Crisis Period

In [9]:
# Create Lists to collect the hedging errors
derivative_payoff = []
error_fix = []
rel_error_fix = []
error_robust = []
epochs = 1000 # Number of epochs for training

for j in range(len(stocks)):        
    print("\n\n ############\n \n ##### Calculations for Stock: {} \n".format(str(j+1)))
    a1 = list_parameters_crisis[j]['a1'][0]
    b1 = list_parameters_crisis[j]['b1'][0]
    x_0 = stocks[j].iloc[start_index]["CLOSE"].astype('float32')
    T = (30/250)
    n = 30
    def derivative(path):
        return tf.nn.relu(10-tf.math.reduce_mean(path))
    #######################################################
    
    price_fix, hedge_fix = optimal_hedge_non_robust(derivative,0,a1,0,b1,1,x_0,T,n,
                             depth = 4,nr_neurons = 256,EPOCHS = epochs,l_r = 0.005,BATCH_SIZE =256,scaling_factor=x_0/10.)



    ####################################################
    #Evaluate
    t_k = np.linspace(0,T,n+1)
    start_index = 2880
    end_index = 2910
    x_0 = stocks[j].iloc[start_index]["CLOSE"].astype('float32')
    test_period= np.array(stocks[j].iloc[start_index:end_index]["CLOSE"])/(x_0/10.) #Normalize the values of the test period

    # Fix Parameters
    hedge_evaluated_fix = [hedge_fix([tf.reshape(test_period[i],(1,1)),tf.reshape(t_k[i],(1,1))]) for i in range(n-1)]
    delta_S_fix = tf.reduce_sum([hedge_evaluated_fix[i]*np.reshape(np.diff(test_period)[i],(1,1)) for i in range(n-1)],0)

    #########################################################
    # Derivative
    derivative_on_batch = derivative(test_period)
    derivative_payoff+= [derivative(test_period)]
    # Compute Hedging Error (Relative), Percentage Values:
    rel_error_fix += [((delta_S_fix.numpy()+price_fix - derivative_on_batch)/price_fix).numpy()]
    error_fix += [10*((delta_S_fix.numpy()+price_fix - derivative_on_batch)).numpy()]



 ############
 
 ##### Calculations for Stock: 1 



  1%|          | 11/1000 [00:05<08:08,  2.02it/s]

Iteration:10, Price of Hedge: 3.3081756521291483, Loss: 1.390904547753405


  2%|▏         | 21/1000 [00:10<08:40,  1.88it/s]

Iteration:20, Price of Hedge: 3.4958253167738804, Loss: 0.9241745091348549


...

 98%|█████████▊| 981/1000 [09:08<00:10,  1.74it/s]

Iteration:980, Price of Hedge: 1.1911613360983095, Loss: 0.08491619917003276


 99%|█████████▉| 991/1000 [09:14<00:05,  1.58it/s]

Iteration:990, Price of Hedge: 1.1858027118007952, Loss: 0.042616691223233256


100%|██████████| 1000/1000 [09:20<00:00,  1.78it/s]


Error

In [10]:
list_computations= pd.DataFrame({'error_fix': [e[0][0] for e in error_fix]})
list_computations.to_csv('csv/sp500_results_asian_atm_bs_crisis.csv',index=False)
np.abs(list_computations).describe()

,error_fix
count,20.000000
mean,10.713926
std,5.367559
min,3.649415
25%,5.861762
50%,9.831267
75%,14.884114
max,21.084524


Relative Error

In [11]:
list_computations= pd.DataFrame({'rel_error_fix': [e[0][0] for e in rel_error_fix]})
list_computations.to_csv('csv/sp500_results_asian_atm_bs_crisis_relative.csv',index=False)
np.abs(list_computations).describe()

,rel_error_fix
count,20.000000
mean,5.785992
std,2.847757
min,2.098309
25%,3.527510
50%,5.412985
75%,7.937476
max,12.002523


## Later Testing window

We do the same procedure, and train the strategy, but evaluate this time every 30 trading days until September 2021. (13 Periods)

In [12]:
# Create Lists to collect the hedging errors
derivative_payoff = []
error_fix = []
rel_error_fix = []
epochs = 1000 # Number of epochs for training

for i in range(1,4):
    print("Period : {}".format(i))
    for j in range(len(stocks)):        
        print("\n\n ############\n \n ##### Calculations for Stock: {} \n".format(str(j+1)))
        a1 = list_parameters[j]['a1'][i-1]
        b1 = list_parameters[j]['b1'][i-1]
        t_k = np.linspace(0,T,n+1)
        start_index = 2880+100*i
        end_index = 2910+100*i
        x_0 = stocks[j].iloc[start_index]["CLOSE"].astype('float32')
        T = (30/250)
        n = 30
        def derivative(path):
            return tf.nn.relu(10-tf.math.reduce_mean(path))
        
        price_fix, hedge_fix = optimal_hedge_non_robust(derivative,0,a1,0,b1,1,x_0,T,n,
                                 depth = 4,nr_neurons = 256,EPOCHS = epochs,l_r = 0.005,BATCH_SIZE =256,scaling_factor=x_0/10.)


        ####################################################
        #Evaluate

        x_0 = stocks[j].iloc[start_index]["CLOSE"].astype('float32')
        test_period= np.array(stocks[j].iloc[start_index:end_index]["CLOSE"])/(x_0/10.) #Normalize the values of the test period

        # Fix Parameters
        hedge_evaluated_fix = [hedge_fix([tf.reshape(test_period[i],(1,1)),tf.reshape(t_k[i],(1,1))]) for i in range(n-1)]
        delta_S_fix = tf.reduce_sum([hedge_evaluated_fix[i]*np.reshape(np.diff(test_period)[i],(1,1)) for i in range(n-1)],0)

        #########################################################
        # Derivative
        derivative_on_batch = derivative(test_period)
        derivative_payoff+= [derivative(test_period)]
        # Compute Hedging Error (Relative), Percentage Values:
        rel_error_fix += [((delta_S_fix.numpy()+price_fix - derivative_on_batch)/price_fix).numpy()]
        error_fix += [10*((delta_S_fix.numpy()+price_fix - derivative_on_batch)).numpy()]

Period : 1


 ############
 
 ##### Calculations for Stock: 1 



  1%|          | 11/1000 [00:06<09:59,  1.65it/s]

Iteration:10, Price of Hedge: 8.180410904789824, Loss: 4.545443990681088


  2%|▏         | 21/1000 [00:13<10:06,  1.62it/s]

Iteration:20, Price of Hedge: 8.442628344122932, Loss: 4.420470837911489


...

 98%|█████████▊| 981/1000 [09:01<00:10,  1.87it/s]

Iteration:980, Price of Hedge: 1.6418659121446126, Loss: 0.03972286633495301


 99%|█████████▉| 991/1000 [09:07<00:04,  1.81it/s]

Iteration:990, Price of Hedge: 1.6371426238098594, Loss: 0.02844086809606239


100%|██████████| 1000/1000 [09:12<00:00,  1.81it/s]


Error

In [13]:
list_computations= pd.DataFrame({'error_fix': [e[0][0] for e in error_fix]})
list_computations.to_csv('csv/sp500_results_asian_atm__bs_rolling_window.csv',index=False)
np.abs(list_computations).describe()

,error_fix
count,60.000000
mean,1.428602
std,0.815293
min,0.026016
25%,0.856307
50%,1.314260
75%,1.990524
max,2.961721


Relative Error

In [14]:
list_computations= pd.DataFrame({'rel_error_fix': [e[0][0] for e in rel_error_fix]})
list_computations.to_csv('csv/sp500_results_asian_atm_bs_rolling_window_relative.csv',index=False)
np.abs(list_computations).describe()

,rel_error_fix
count,60.000000
mean,0.462730
std,0.229142
min,0.017003
25%,0.334309
50%,0.501456
75%,0.614696
max,0.861197
